In [2]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform

In [11]:
# Functions

# Get panel masks

def get_panel_masks(geojson_path, ref_img_path):
    '''This function generates a mask that leaves only pixels inside 
    vine panels unmasked. 
    
    Inputs:
    geojson_path (str): path to a geojson file with the panel geometries. 
    Must be in same CRS as image, otherwise un-comment code section starting with "dst_crs".
    ref_img_path (str): path to the .tif file for the image
    
    Outputs:
    A list of masks, as Boolean arrays. True values are masked pixels.
    False values are unmasked pixels.
    '''
    with open(geojson_path, 'r') as f:
        panels = json.load(f)
        features = panels['features']
        features_ordered = sorted(features, key= lambda feature:(feature['properties']['row'],feature['properties']['panel'])) # ground truth order.
        panel_aois = [f['geometry'] for f in features_ordered]
    
    with rio.open(ref_img_path) as src:
        dst_crs = src.crs
        
        # geojson features (the field block geometries)
        # are often in WGS84
        # project these to the image coordinates
        wgs84 = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
        proj_panel_aois = [transform(project, shape(p)) for p in panel_aois]

        masks = [raster_geometry_mask(src, [p], crop=False)[0]
                 for p in proj_panel_aois]                       
    return masks

# Get reflectance df

def get_rfl(img_path, masks):
    '''
    Function to extract per-panel reflectance values for 4-band SkySat images.
    Calculates the mean reflectance values for the pixels in each panel.
    The output is formatted as a pandas dataframe.
    
    Inputs:
    img_path - abs. path to the image       
    masks - a list of masks that correspond to the panel aois. 
    '''
    
    with rio.open(img_path,'r') as img:
        def _get_stats(mask, panel_number):
            panel_blue = np.ma.mean(np.ma.array(img.read(1), mask=mask))/10000
            panel_green = np.ma.mean(np.ma.array(img.read(2), mask=mask))/10000
            panel_red = np.ma.mean(np.ma.array(img.read(3), mask=mask))/10000
            panel_nir = np.ma.mean(np.ma.array(img.read(4), mask=mask))/10000
            
            return {'acquired': img_path.split('/')[-1].split('_')[-5], # date from image filename
                    'plot': panel_number + 1,    #add one - panels are numbered starting at 1 in the 'real world'
                    'blue': panel_blue,
                    'green': panel_green,
                    'red': panel_red,
                    'nir': panel_nir}
        data = [_get_stats(m, i) for i, m in enumerate(masks)]  # sets the parameters for the nested function as the mask (m) and corresponding panel number (i). 
        df = pd.DataFrame(data)
    
    return df

In [12]:
# image directory containing co-registered SkySat scenes 

img_dir = '../data/images/2021/SkySat/'

# image pathnames
img_paths = glob.glob(img_dir+'2021*.tif')

# geojson with panel geometries

json_path = '../data/crit_panels_geojson.geojson'
#json_path
img_paths

['../data/images/2021/SkySat/20210726_160338_ssc1d2_0013_ar.tif',
 '../data/images/2021/SkySat/20210802_184019_ssc10d2_0005_ar.tif',
 '../data/images/2021/SkySat/20210707_150122_ssc19d2_0016_ar.tif',
 '../data/images/2021/SkySat/20210816_171007_ssc18d2_0015_ar.tif',
 '../data/images/2021/SkySat/20210809_185329_ssc9d2_0014_ar.tif']

In [13]:
# Get dataframes for all images in directory

master_df = pd.DataFrame()

for i in img_paths:
    masks = get_panel_masks(json_path, i)
    df = get_rfl(i, masks)
    master_df = pd.concat([master_df, df], axis=0)
    master_df = master_df.sort_values(by=['acquired', 'plot'])

master_df

,acquired,plot,blue,green,red,nir
0,20210707,1,0.032006,0.066432,0.040084,0.290003
1,20210707,2,0.033528,0.067879,0.043324,0.270290
2,20210707,3,0.033903,0.067897,0.042947,0.273817
3,20210707,4,0.032868,0.067358,0.042055,0.280455
4,20210707,5,0.032104,0.066636,0.038836,0.292664
...,...,...,...,...,...,...
315,20210816,316,0.016079,0.064390,0.045383,0.246759
316,20210816,317,0.012997,0.062097,0.043137,0.247093
317,20210816,318,0.012376,0.061769,0.041824,0.254628
318,20210816,319,0.011006,0.059225,0.038784,0.253509
